# Import Libraries

In [ ]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
!pip install -q torch-geometric

In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)

import numpy as np
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import normalized_mutual_info_score as nmi, adjusted_rand_score as ari
from sklearn.cluster import KMeans
import random
import warnings
warnings.filterwarnings('ignore')

import tqdm
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid, Reddit, Yelp
from torch_geometric.data import Data

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# SKFR1

In [ ]:
# implementing z score to normalize each feature using mean and std via torch
def zscore(x):
    '''
    zscore normalization
    '''
    # get mean and std of each feature
    mean = torch.mean(x)
    std = torch.std(x)
    # if std is not 0, then normalize each feature
    if std != 0:
        x = (x - mean) / std
    else:
        x = x - mean
    return x



In [ ]:
# PyTorch Implementation of the SKFR1 algorithm
# Input: data of shape (features, samples), clusters, k (number of clusters), sparsity
# Output: center of shape (features, clusters), clusters, list_loss
def skfr1(data, clusters, k, sparsity, maxiter):
    '''
    SKFR1 algorithm
    '''
    # get number of features and samples
    features, samples = data.shape
    # criteria of shape (features)
    criteria = torch.zeros(features).to(device)

    # normalize each feature
    for i in range(features):
        data[i, :] = zscore(data[i, :])
    
    # list_loss empty list
    list_loss = []
    # number of iterations
    num_iter = 1
    switched = True

    while switched and num_iter < maxiter:
        # initialize clusters
        center = torch.zeros(features, k).to(device)
        # members of size clusters
        members = torch.zero(k).to(device)
        # for each sample
        for j in range(samples):
            # i as jth cluster 
            i = clusters[j]
            # add sample to center
            center[:, i] = center[:, i] + data[:, j]
            # add 1 to members
            members[i] = members[i] + 1
        # for each cluster
        for j in range(k):
            # if members is not zero
            if members[j] != 0:
                # divide by members
                center[:, j] = center[:, j] / members[j]
        # update criteria d_l
        criteria = torch.matmul(torch.mul(center, center), members.T)
        # get index from criteria
        index = torch.LongTensor([i for i in range(len(criteria))]).to(device)
        # sort criteria
        sorted_criteria = sorted(zip(criteria, index))
        J = [x[1] for x in sorted_criteria]
        # make long tensor of J
        J = torch.LongTensor(J).to(device)
        # get only features-sparsity features of J
        J = J[:features-sparsity]
        # for each J feature index
        for i in range(len(J)):
            center[J[i]] = torch.zeros(k).to(device)
        # deleter members, criteria, index, sorted_criteria, J
        del members, criteria, index, sorted_criteria, J

        # get distamce as square root of sum of square of each feature 
        distance = torch.sqrt(((X.T - center.T[:, np.newaxis])**2).sum(axis=2))
        switched = False
        # for each sample
        for i in range(samples):
            # get index of minimum distance
            j = torch.argmin(distance[:, i])
            # if this index is not same as cluster
            if clusters[i] != j:
                # update cluster
                clusters[i] = j
                switched = True
        # deleter distance
        del distance
        # WSS as sum of square of each feature. Initialize to zero
        WSS = torch.zeros(k).to(device)
        # for each cluster
        for k in range(k):
            # get temporary index where cluster is k
            temp_index = torch.LongTensor(np.where(clusters.cpu().numpy() == k)[0]).to(device)
            # tempX as zero tensor of shape (features, len(temp_index))
            tempX = torch.zeros(features, len(temp_index)).to(device)
            # for each temp_index
            for j in range(len(temp_index)):
                # add data to tempX
                tempX[:, j] = data[:, temp_index[j]]
            # get WSS
            WSS[k] = torch.mean(((tempX.T - center[:, k]).T)**2)
        # get loss as sum of WSS
        loss = torch.sum(WSS)
        # add loss to list_loss
        list_loss.append(loss.item())
        # print iteration number and loss
        print('Iteration: {} Loss: {}'.format(num_iter, loss.item()))
        # deleter tempX, temp_index, WSS, loss
        del tempX, temp_index, WSS, loss
        # update iteration number
        num_iter += 1
    return center, clusters, list_loss


# SKFR2

In [ ]:
# PyTorch Implementation of the SKFR2 algorithm
# Input: data of shape (features, samples), clusters, k (number of clusters), sparsity
# Output: center of shape (features, clusters), clusters, list_loss
def skfr2(data, clusters, k, sparsity, maxiter):
    '''
    SKFR2 algorithm
    '''
    # get number of features and samples
    features, samples = data.shape
    # criteria of shape (features)
    criteria = torch.zeros(features).to(device)

    # normalize each feature
    for i in range(features):
        data[i, :] = zscore(data[i, :])
    
    # list_loss empty list
    list_loss = []
    # number of iterations
    num_iter = 1
    switched = True

    while switched and num_iter < maxiter:
        # initialize clusters
        center = torch.zeros(features, k).to(device)
        # members of size clusters
        members = torch.zero(k).to(device)
        # for each sample
        for j in range(samples):
            # i as jth cluster 
            i = clusters[j]
            # add sample to center
            center[:, i] = center[:, i] + data[:, j]
            # add 1 to members
            members[i] = members[i] + 1
        # for each cluster
        for j in range(k):
            # if members is not zero
            if members[j] > 0:
                # divide by members
                center[:, j] = center[:, j] / members[j]
                # Get criteria = number of members in cluster multiplied by center*center
                criteria = members[j] * torch.mul(center[:, j], center[:, j])
                # get index from criteria
                index = torch.LongTensor([i for i in range(len(criteria))]).to(device)
                # sort criteria
                sorted_criteria = sorted(zip(criteria, index))
                J = [x[1] for x in sorted_criteria]
                # make long tensor of J
                J = torch.LongTensor(J).to(device)
                # get only features-sparsity features of J
                J = J[:features-sparsity]
                for i in range(len(J)):
                    center[J[i], j] = 0
        # deleter members, criteria, index, sorted_criteria, J
        del members, criteria, index, sorted_criteria, J

        # get distamce as square root of sum of square of each feature 
        distance = torch.sqrt(((X.T - center.T[:, np.newaxis])**2).sum(axis=2))
        switched = False
        # for each sample
        for i in range(samples):
            # get index of minimum distance
            j = torch.argmin(distance[:, i])
            # if this index is not same as cluster
            if clusters[i] != j:
                # update cluster
                clusters[i] = j
                switched = True
        # deleter distance
        del distance
        # WSS as sum of square of each feature. Initialize to zero
        WSS = torch.zeros(k).to(device)
        # for each cluster
        for k in range(k):
            # get temporary index where cluster is k
            temp_index = torch.LongTensor(np.where(clusters.cpu().numpy() == k)[0]).to(device)
            # tempX as zero tensor of shape (features, len(temp_index))
            tempX = torch.zeros(features, len(temp_index)).to(device)
            # for each temp_index
            for j in range(len(temp_index)):
                # add data to tempX
                tempX[:, j] = data[:, temp_index[j]]
            # get WSS
            WSS[k] = torch.mean(((tempX.T - center[:, k]).T)**2)
        # get loss as sum of WSS
        loss = torch.sum(WSS)
        # add loss to list_loss
        list_loss.append(loss.item())
        # print iteration number and loss
        print('Iteration: {} Loss: {}'.format(num_iter, loss.item()))
        # deleter tempX, temp_index, WSS, loss
        del tempX, temp_index, WSS, loss
        # update iteration number
        num_iter += 1
    return center, clusters, list_loss







